In [ ]:
!sudo apt-get update -qq

In [ ]:
!sudo apt-get install -y p7zip

In [17]:
import numpy as np
import pandas as pd

## Descarga de los datos

En este caso los datos están disponibles en un repositorio git. Se pueden descargar también de la Web, pero se van actualizando. Los descargamos del repositorio git para que todos tengáis los mismos.

In [ ]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.002
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.003

In [ ]:
!ls -lh es.stackoverflow.7z*

In [ ]:
!7zr x es.stackoverflow.7z.001
!rm es.stackoverflow.7z.*

In [ ]:
!ls -lh *.xml

## Inspección y procesado

Podemos inspeccionar los ficheros `.xml` para ver su contenido. Son XML, sí, pero ¿con qué formato?

Aunque se puede procesar el formato XML, lo que podemos ver es que cada entrada es exactamente una línea que comienza por "`<row`", y que contiene un conjunto de atributos en formato "`atributo="valor"`". Si lo comprobamos, incluso no existirá ninguna comilla doble **dentro** de otra comilla doble, así que podemos extraer esos pares de forma facil.

La siguiente función procesa el fichero XML línea a línea. Primero separa la parte inicial "`<row`", y después procesa cada par clave/valor. Lo único que hace es construir el conjunto de atributos que hay en todas las entradas. Como vimos, cada fila contenía atributos diferentes. Queremos obtenerlos todos.

La función, en vez de retornar una lista, que ocuparía mucha memoria, retorna un generador, que es una lista (de pares clave-valor, un diccionario) que se va generando a medida que se recorre. Por eso utiliza la construcción `yield` de Python. Esto hace que la función se detenga, y cuando se le pide el siguiente elemento, continúa desde donde se quedó (corrutina).

In [8]:
import re
from typing import Iterator

def generate_elements_from_lines(filename: str) -> Iterator[dict[str, str]]:

  def get_attrs(line: str) -> dict[str, str]:
    (_, attrs) = line.split("<row ", 2)
    return {m.group(1): m.group(2)
              for m in re.finditer(r"(\w*?)=\"(.*?)\"", attrs)}

  with open(filename, "r") as f:
    for line in f:
      if "<row" in line:
        yield get_attrs(line)

Hay que extraer el conjunto de atributos para saber qué columnas tendrá nuestra tabla/CSV o archivo JSON. Recuérdese que las dos primeras filas del archivo XML tenían diferentes atributos. ¿Cómo se haría esto?

In [9]:
from typing import Iterator

def get_all_attrs(iterator: Iterator[dict[str,str]]) -> set[str]:
  all_attrs: set[str] = set()
  for row in iterator:
    all_attrs.update(row.keys())
  return all_attrs

El conjunto de atributos es pues:

In [ ]:
all_attrs_posts = get_all_attrs(generate_elements_from_lines("Posts.xml"))
all_attrs_posts

Como sabemos que el atributo `Id` va a ser la clave primaria, lo ponemos al principio. Además, generamos una lista, no un conjunto, para que el orden sea conocido.

In [11]:
def id_as_first_attribute(all_attrs: set[str], id_name: str) -> list[str]:
  all_attrs.remove(id_name)
  return [id_name] + list(all_attrs)

In [ ]:
all_attrs_posts = id_as_first_attribute(all_attrs_posts, 'Id')
all_attrs_posts

In [ ]:
%pip install pyarrow

## Escritura del formato CSV

El formato CSV está especificado en el estándar RFC 4180. https://www.ietf.org/rfc/rfc4180.txt. En general se puede utilizar la biblioteca `csv` de Python 3 y vamos a exportar una línea de cabecera con todos los campos. https://docs.python.org/3/library/csv.html.

Tendremos en cuenta que todas las filas tienen que tener las mismas columnas y en el mismo orden dado por `all_attrs`.

In [14]:
import csv

def write_csv(destfile: str, all_attrs: list[str], iterator: Iterator[dict[str,str]]) -> None:
  with open(destfile, 'w') as wf:
    cw = csv.writer(wf)

    # Escribir la línea de cabecera
    cw.writerow(all_attrs)

    # Recorrer el iterador
    for row in iterator:
      row_to_write = [row.get(att, '') for att in all_attrs]
      cw.writerow(row_to_write)

In [15]:
write_csv('Posts.csv', all_attrs_posts, generate_elements_from_lines('Posts.xml'))

In [18]:
# Write the df dataframe to parquet file
df = pd.read_csv('Posts.csv', encoding='utf-8', header=0,
                 dtype={'Id': 'Int64', 'PostTypeId': 'Int64', 'AcceptedAnswerId': 'Int64', 'ParentId': 'Int64',
                        'Score': 'Int64', 'ViewCount': 'Int64',
                        'Body': pd.StringDtype(), 'OwnerUserId': 'Int64', 'OwnerDisplayName': pd.StringDtype(),
                        'LastEditorUserId': 'Int64', 'LastEditorDisplayName': pd.StringDtype(),
                        'Title': pd.StringDtype(), 'Tags': pd.StringDtype(), 'AnswerCount': 'Int64',
                        'CommentCount': 'Int64', 'FavoriteCount': 'Int64',
                        'ContentLicense': pd.StringDtype()},
                 parse_dates=['CreationDate','LastEditDate','LastActivityDate','ClosedDate','CommunityOwnedDate'])

In [ ]:
df.head()

In [20]:
df.to_parquet('Posts.parquet', compression='snappy')


In [ ]:
all_attrs_votes = get_all_attrs(generate_elements_from_lines("Votes.xml"))
all_attrs_votes

In [ ]:
all_attrs_votes = id_as_first_attribute(all_attrs_votes, 'Id')
all_attrs_votes

In [24]:
write_csv('Votes.csv', all_attrs_votes, generate_elements_from_lines('Votes.xml'))

In [25]:
# Write the df dataframe to parquet file
df = pd.read_csv('Votes.csv', encoding='utf-8', header=0,
                 dtype={'Id': 'Int64', 'VoteTypeId' : 'Int64', 'BountyAmount' : 'Int64', 'PostId': 'Int64', 'UserId' : 'Int64' },
                 parse_dates=['CreationDate'])

In [ ]:
df.head()

In [26]:
df.to_parquet('Votes.parquet', compression='snappy')

In [ ]:
all_attrs_votes = get_all_attrs(generate_elements_from_lines("Votes.xml"))
all_attrs_votes

In [ ]:
all_attrs_votes = id_as_first_attribute(all_attrs_votes, 'Id')
all_attrs_votes

In [ ]:
write_csv('Votes.csv', all_attrs_votes, generate_elements_from_lines('Votes.xml'))

In [ ]:
# Write the df dataframe to parquet file
df = pd.read_csv('Votes.csv', encoding='utf-8', header=0,
                 dtype={'Id': 'Int64', 'VoteTypeId' : 'Int64', 'BountyAmount' : 'Int64', 'PostId': 'Int64', 'UserId' : 'Int64' },
                 parse_dates=['CreationDate'])

In [ ]:
df.head()

In [ ]:
df.to_parquet('Votes.parquet', compression='snappy')